In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
from refractio import snowflake
snowflake.get_connection(connection_name="HR_ATTRITION")

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Exception occurred in getting snowflake connection: 'connectionSources'


In [3]:
df = snowflake.get_dataframe("HR_ATTRITION")
df

,SALARY,EMPLOYEE_ID,JOB_SATISFACTION,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,OVER_TIME,RELATIONSHIP_SATISFACTION,CHURN_STATUS_TABLE,CHURN_VALUE_TABLE,CITY,...,RETENTION,SUM_OF_TENURE,SUM_OF_AGE,JOB_STARTDATE,JOB_ENDDATE,OVERTIME_HOURS,WORK_LIFE_BALANCE,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,USER_ID
0,43118.65,94215814,High,20,Outstanding,No,Medium,FALSE,0,Charlotte,...,0,0,0,2016-06-01,2019-08-01,4,Best,Travel Rarely,Medium,334493477
1,40808.18,72815065,Very High,12,Low,No,Very High,TRUE,1,Charlotte,...,0,0,0,2015-05-01,None,7,Best,Travel Frequently,Low,713161033
2,36394.69,14265310,Low,13,Good,No,Medium,FALSE,0,Pittsburgh,...,0,0,0,2017-06-01,2018-07-01,8,Good,Travel Rarely,High,154131612
3,96880.66,38467325,Low,21,Good,No,Very High,FALSE,0,Dallas,...,0,0,0,2019-09-01,2024-07-01,14,Bad,Travel Rarely,Very High,518711452
4,97579.66,98539130,High,17,Good,No,Low,FALSE,0,Dallas,...,0,0,0,2019-09-01,2024-08-01,9,Best,Travel Frequently,Low,518711452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,101713.50,66752547,Low,17,Good,No,Very High,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-06-01,1,Good,Travel Rarely,Low,492160448
299996,101640.50,61579487,High,13,Excellent,No,Very High,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-06-01,1,Best,Travel Frequently,Low,492160448
299997,101998.50,48056583,Very High,21,Outstanding,No,Medium,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-06-01,1,Good,Travel Rarely,Low,492160448
299998,101306.50,86413761,Medium,20,Low,No,Low,FALSE,0,Houston,...,609,37548,9563,2021-12-01,2023-06-01,1,Best,Travel Frequently,Medium,492160448


In [4]:
print(df.isnull().sum())

SALARY                                             0
EMPLOYEE_ID                                        0
JOB_SATISFACTION                                   0
PERCENTAGE_SALARY_HIKE                             0
PERFORMANCE_RATING                                 0
OVER_TIME                                          0
RELATIONSHIP_SATISFACTION                          0
CHURN_STATUS_TABLE                                 0
CHURN_VALUE_TABLE                                  0
CITY                                               0
DISTANCE                                           0
DEGREE_CLEAN                                       0
ETHNICITY                                          0
MARITAL_STATUS                                     0
ROLE                                               0
COMPANY                                            0
ORGANIZATION_TYPE                                  0
ORGANIZATION_OWNERSHIP                             0
STATE                                         

In [5]:
Original_df = df.dropna()

In [6]:
print(Original_df.isnull().sum())

SALARY                                         0
EMPLOYEE_ID                                    0
JOB_SATISFACTION                               0
PERCENTAGE_SALARY_HIKE                         0
PERFORMANCE_RATING                             0
OVER_TIME                                      0
RELATIONSHIP_SATISFACTION                      0
CHURN_STATUS_TABLE                             0
CHURN_VALUE_TABLE                              0
CITY                                           0
DISTANCE                                       0
DEGREE_CLEAN                                   0
ETHNICITY                                      0
MARITAL_STATUS                                 0
ROLE                                           0
COMPANY                                        0
ORGANIZATION_TYPE                              0
ORGANIZATION_OWNERSHIP                         0
STATE                                          0
COUNTRY                                        0
GENDER              

In [22]:
df = Original_df.drop(["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "CHURN_STATUS_TABLE","SCHOOL_ENDDATE","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","POPULATION","CHURN_OTHER","RETENTION","SUM_OF_TENURE","SUM_OF_AGE"], axis = 1)

In [25]:
CATEGORICAL_COLUMNS = ["ROLE","GENDER", "ETHNICITY","ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP","COMPANY","CITY","STATE","COUNTRY","DISTANCE",
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARITAL_STATUS","OVER_TIME","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE","OVERTIME_HOURS","PERCENTAGE_SALARY_HIKE"]
LABEL_COLUMNS = ["CHURN_VALUE_TABLE "]
DROPPED_COLUMNS = ["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "CHURN_STATUS_TABLE","SCHOOL_ENDDATE","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","POPULATION","CHURN_OTHER","RETENTION","SUM_OF_TENURE","SUM_OF_AGE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [26]:
# Verify filtered label columns
print("Label columns:", LABEL_COLUMNS)

Label columns: ['CHURN_VALUE_TABLE ']


In [31]:
# Filter feature columns
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS + DROPPED_COLUMNS
feature_columns = [col for col in feature_columns if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]
 
# Split data into features and labels
X = df[feature_columns]
y = df[LABEL_COLUMNS].values.ravel()  # Flatten to 1D array for consistency

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [209924, 0]

In [34]:
# Verify feature columns
print("Feature columns:", feature_columns)

Feature columns: ['ROLE', 'GENDER', 'ETHNICITY', 'ORGANIZATION_TYPE', 'ORGANIZATION_OWNERSHIP', 'COMPANY', 'CITY', 'STATE', 'COUNTRY', 'DISTANCE', 'DEGREE_CLEAN', 'BUSINESS_TRAVEL', 'ENVIRONMENT_SATISFACTION', 'JOB_SATISFACTION', 'MARITAL_STATUS', 'OVER_TIME', 'PERFORMANCE_RATING', 'RELATIONSHIP_SATISFACTION', 'WORK_LIFE_BALANCE', 'SALARY', 'SENIORITY', 'TENURE_MONTHS', 'MONTHS_AFTER_COLLEGE', 'BIRTH_YEAR', 'AGE', 'OVERTIME_HOURS', 'PERCENTAGE_SALARY_HIKE']


In [30]:
# Verify filtered label columns
print("Label columns:", LABEL_COLUMNS)

Label columns: []
